### 네이버 금융 정보 가져오기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import time
import re

import pandas as pd
import xlsxwriter

In [2]:
from selenium import webdriver
driver = webdriver.Chrome('chromedriver')

In [3]:
## 페이지 이동
url = "https://finance.naver.com/sise/"
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source, 'lxml')
soup.title

<title>국내증시 : 네이버 금융</title>

In [5]:
kospi = soup.find("span", id="KOSPI_now").text
kosdaq = soup.find("span", id="KOSPI_now").text
kospi_200 = soup.find("span", id="KPI200_now").text
print("코스피 지수 : {}".format(kospi))
print("코스닥 지수 : {}".format(kosdaq))
print("코스피200 지수 : {}".format(kospi_200))

코스피 지수 : 2,390.47
코스닥 지수 : 2,390.47
코스피200 지수 : 317.83


In [6]:
pop_search = soup.find("ul", class_="lst_pop")
pop_search_li = pop_search.find_all("li")
pop_search_li[0]

<li><em>1.</em><a href="/item/main.nhn?code=042040" onclick="clickcr(this,'boa.list','042040','1',event)">케이피엠테..</a><span class="up">4,845</span><img alt="상승" height="6" src="https://ssl.pstatic.net/static/nfinance/ico_up.gif" width="7"/></li>

In [7]:
all_stock = []
all_stock_url = []
code_all_num = []

for one in pop_search_li:
    stock = one.find("a").text
    price = one.find("span").text
    all_stock.append(stock)             # 종목명
    all_stock_url.append(price)         # 가격
    
    code = one.find("a").attrs
    code_num = code['onclick']         
    code_all = re.split(",", code_num)   
    code_all_num.append(code_all[2])    # 종목 코드

print( all_stock )
print( all_stock_url )
print( code_all_num )

['케이피엠테..', '한솔홈데코', '삼성전자', '씨젠', '에코바이오', 'SK텔레콤', '엑세스바이..', 'LG화학', 'SK바이오팜', '신풍제약']
['4,845', '2,145', '60,800', '288,400', '11,750', '230,500', '34,550', '627,000', '155,500', '128,000']
["'042040'", "'025750'", "'005930'", "'096530'", "'038870'", "'017670'", "'950130'", "'051910'", "'326030'", "'019170'"]


In [8]:
dat = {"종목":all_stock, "가격":all_stock_url, "종목명":code_all_num}
pop_stock = pd.DataFrame(dat)
pop_stock.to_excel("stock.xlsx", sheet_name="Top10", index=False)

In [9]:
pop_search = soup.find("ul", class_="lst_major")
pop_search_li = pop_search.find_all("li")
pop_search_li[0]

<li><a href="/world/sise.nhn?symbol=DJI@DJI&amp;fdtc=0" onclick="clickcr(this,'wst.dow','','',event)">다우산업</a><span class="dn">28,679.81</span><em class="bu_p bu_pdn"><span class="blind">하락</span></em></li>

In [10]:
all_stock = []
all_stock_price = []
code_all_num = []
for one in pop_search_li:
    stock = one.find("a").text
    price = one.find("span").text
    all_stock.append(stock)             # 종목명
    all_stock_price.append(price)         # 가격
    
print( all_stock )
print( all_stock_price )

['다우산업', ' 나스닥', ' 홍콩H', ' 상해종합', ' 니케이225']
['28,679.81', '11,863.90', '9,894.12', '3,344.81', '23,589.35']


In [11]:
dat = {"해외지수":all_stock, "지수":all_stock_price}
pop_stock = pd.DataFrame(dat)
pop_stock

,해외지수,지수
0,다우산업,"28,679.81"
1,나스닥,"11,863.90"
2,홍콩H,"9,894.12"
3,상해종합,"3,344.81"
4,니케이225,"23,589.35"


In [12]:
dat1 = pd.read_excel("stock.xlsx", sheet_name="Top10")

In [13]:
import openpyxl
workbook1 = openpyxl.load_workbook("stock.xlsx")
print(workbook1.sheetnames)

# 시트 삭제
Sheet1 = workbook1['Top10']
workbook1.remove(Sheet1)

['Top10']


In [14]:
writer = pd.ExcelWriter("stock.xlsx", engine='xlsxwriter')

dat1.to_excel(writer, sheet_name="Top10", index=False)
pop_stock.to_excel(writer, sheet_name="해외지수", index=False)

writer.save()  # 엑셀 파일 저장